<a href="https://colab.research.google.com/github/PortugalJip2022/Portugal2022/blob/main/%E6%89%8B%E6%9B%B8%E3%81%8D%E6%96%87%E5%AD%97%E3%83%81%E3%83%A5%E3%83%BC%E3%83%88%E3%83%AA%E3%82%A2%E3%83%AB_%E6%B0%B4%E8%B0%B7_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN　具体的に何してる？？  
・「事前準備」と「チュートリアル実行」は何も考えずそのまま実行  
・「テスト用」から、具体的に何してる？を実行